# Data processing with Pandas, part 2

This week we will continue developing our skills using [Pandas](https://pandas.pydata.org/) to process real data. 

## Motivation

![Finland April 2019](img/Finland-April-2019.png)
*Source: [https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa)*

April 2019 was the [second warmest April on record globally](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa), and the warmest on record at 13 weather stations in Finland. 
In this lesson, we will use our data manipulation and analysis skills to analyze weather data from Finland, and investigate the claim that April 2019 was the warmest on record across Finland.

Along the way we will cover a number of useful techniques in pandas including:

- renaming columns
- iterating data frame rows and applying functions
- data aggregation
- repeating the analysis task for several input files 

## Input data

In the lesson this week we are using weather observation data from Finland [downloaded from NOAA](https://www7.ncdc.noaa.gov/CDO/cdopoemain.cmd?datasetabbv=DS3505&countryabbv=&georegionabbv=&resolution=40). You will be working with data from either 15 or 4 different weather obsercation stations from Finland, depending on your environment. 

## Downloading the data

The first step for today's lesson is to get the data. Which data files you download will depend on the platform you're using for working through the lesson today.

### CSC Notebooks users

If you're working on the CSC Notebooks platform, you can download the data by opening a new terminal window in Jupyter Lab by going to **File** -> **New** -> **Terminal** in the Jupyter Lab menu bar. Once the terminal is open, you will need to navigate to the directory for Lesson 6 by typing

```bash
cd notebooks/notebooks/L6/
```

Once in the correct directory, you can confirm this by typing

```bash
ls
```

You should see something like the following output:

```bash
advanced-data-processing-with-pandas.ipynb errors.ipynb                               img
debugging.ipynb                            gcp-assertions.ipynb
```

If so, you're in the correct directory and you can download the data files by typing

```bash
wget https://davewhipp.github.io/data/Finland-weather-data-CSC.tar.gz
```

After the download completes, you can extract the data files by typing

```bash
tar zxvf Finland-weather-data-CSC.tar.gz
```

At this stage you should have a new directory called `data` that contains the data for this week's lesson. You can confirm this by typing

```bash
ls data
```

You should see something like the following:

```bash
029440.txt           029720.txt           3505doc.txt          6367598020644stn.txt
029700.txt           029740.txt           6367598020644inv.txt
```

Now you should be all set to proceed with the lesson!

### Users with Jupyter on their personal computers

If you're working on your own computer, you can download the data by opening a new terminal window in Jupyter Lab by going to **File** -> **New** -> **Terminal** in the Jupyter Lab menu bar. Once the terminal is open, you will need to navigate to the directory for Lesson 6 by typing

```bash
cd path/to/L6/
```

where `path/to/` should be replaced with the directories needed to locate the Lesson 6 materials in on your computer. Once in the correct directory, you can confirm this by typing

```bash
ls
```

You should see something like the following output:

```bash
advanced-data-processing-with-pandas.ipynb errors.ipynb                               img
debugging.ipynb                            gcp-assertions.ipynb
```

If so, you're in the correct directory and you can download the data files by typing

```bash
wget https://davewhipp.github.io/data/Finland-weather-data-full.tar.gz
```

After the download completes, you can extract the data files by typing

```bash
tar zxvf Finland-weather-data-full.tar.gz
```

At this stage you should have a new directory called `data` that contains the data for this week's lesson. You can confirm this by typing

```bash
ls data
```

You should see something like the following:

```bash
028360.txt           029070.txt           029440.txt           029740.txt  6367598020644inv.txt
028690.txt           029110.txt           029500.txt           029810.txt  6367598020644stn.txt
028750.txt           029170.txt           029700.txt           029820.txt
028970.txt           029350.txt           029720.txt           3505doc.txt
```

Now you should be all set to proceed with the lesson!

### Binder users

It is not recommended to complete this lesson using Binder.

## About the data

As part of the download there are a number of files that describe the weather data. These *metadata* files include:

- A list of stations\*: [data/6367598020644stn.txt](metadata/6367598020644stn.txt)
- Details about weather observations at each station: [data/6367598020644inv.txt](metadata/6367598020644inv.txt)
- A data description (i.e., column names): [data/3505doc.txt](metadata/3505doc.txt)

\*Note that the list of stations is for all 15 stations, even if you're working with only the 4 stations on the CSC Notebooks platform.

The input data for this week are separated with varying number of spaces (i.e., fixed width). The first lines and columns of the data look like following:

``` 
  USAF  WBAN YR--MODAHRMN DIR SPD GUS CLG SKC L M H  VSB MW MW MW MW AW AW AW AW W TEMP DEWP    SLP   ALT    STP MAX MIN PCP01 PCP06 PCP24 PCPXX SD
029440 99999 190601010600 090   7 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1011.0 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601011300 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1015.5 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601012000 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   25 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601020600 ***   0 *** *** CLR * * *  0.0 ** ** ** ** ** ** ** ** *   26 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** **
```

We will develop our analysis workflow using data for a single station. Then, we will repeat the same process for all the stations.

## Reading the data

In order to get started, let's import pandas: 

At this point, you can already had a quick look at the input file `029440.txt` for Tampere Pirkkala and how it is structured. We can notice at least two things we need to consider when reading in the data:

<div class="alert alert-info">

**NOTE: Input data structure**

- **Delimiter:** The data are **separated with varying amount of spaces**. If you check out the documentation for the [read_csv() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), you can see that there are two different ways of doing this. We can either use the `sep` or `delim_whitespace` parameter;  `sep='\s+'` or `delim_whitespace=True` but not both. In this case, we prefer to use `delim_whitespace`.

- **No Data values:** No data values in the NOAA data are coded with varyingg number of `*`. We can tell pandas to consider those characters as NaNs by specifying `na_values=['*', '**', '***', '****', '*****', '******']`.
</div>

In [ ]:
fp = r"data/029440.txt"

# Read data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, na_values=['*', '**', '***', '****', '*****', '******'])

- Let's see how the data looks by printing the first five rows with the `head()` function

All seems ok. However, we won't be needing all of the 33 columns for detecting warm temperatures in April. We can check all column names by running `data.columns`:

A description for all these columns is available in the metadata file [data/3505doc.txt](metadata/3505doc.txt). 

**Let's read in the data one more time.** This time, we will read in only some of the columns using the `usecols` parameter. Let's read in columns that might be somehow useful to our analysis, or at least that contain some values that are meaningful to us, including the station name, timestamp, and data about wind and temperature: `'USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'`

In [ ]:
data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])
data.head()

Okay so we can see that the data was successfully read to the DataFrame and we also seemed to be able to convert the asterisk (\*) characters into `NaN` values. 

## Renaming columns

Check again the column names in our DataFrame:

The column names that we have are somewhat ackward. Let's change them into more intuitive ones. 
This can be done easily using the `rename()` method and a dictionary that lists old and new column names.

<div class="alert alert-info">

**Dictionaries**

[Dictionary](https://docs.python.org/2/tutorial/datastructures.html#dictionaries) is a spesific data structure in Python for storing key-value pairs. During this course, we will use dictionaries mainly when renaming columns in a pandas series, but dictionaries are useful for many different purposes! For more information about Python dictionaries, check out [this tutorial](https://realpython.com/python-dicts/).
    
</div>



We can define the new column names using a [dictionary](https://www.tutorialspoint.com/python/python_dictionary.htm) where we determine "`key: value`" -pairs, in which the original column name (the one which will be replaced) is the key, and the new column name is the value.

- Let's change:
   
   - `YR--MODAHRMN` column into `TIME`, 
   - `SPD` into `SPEED`, and
   - `GUS` into `GUST`

In [ ]:
# Create the dictionary with old and new names


# Let's see what they look like and what is the type


From above we can see that we have successfully created a dictionary that is of type `dict`. 

- Now we can change the column names by passing that dictionary into parameter `columns` in `rename()` -function.

In [ ]:
# Rename the columns


# Print the new columns
print(data.columns)

Perfect, now our column names are more easy to understand and use. 

<div class="alert alert-info">

**TASK: Renaming columns**

The temperature values are again in Fahrenheit. As you might guess, we will soon convert these temperatures in to Celsius. In order to avoid confusion with the columns, rename column `TEMP` into `TEMP_F`. Also, we could rename `USAF` as`STATION_NUMBER`.
</div>


In [ ]:
# Create the dictionary with old and new names


# Rename the columns


# Check the output


## Check dataframe properties

As we learned last week, it's always a good idea to check basic properties of the input data before proceeding with data analysis. Let's check:

- How many rows and columns we have:

- Top and bottom rows: 

- Data types of the columns: 

- Descriptive statistics:

Here we can see that there are varying number of observations per column (see the `count` information), because some of the columns have missing values.

## Using your own functions in pandas 

Now it's again time to convert temperatures from Fahrenheit to Celsius! Yes, we have already done this many times before, but this time we will learn how to apply self-made functions to data in a pandas DataFrame.
**In short, our task is to define a function for the temperature conversion, and to apply this function for each Celsius value on each row of the DataFrame. Output celsius values should be stored in a new column called** `TEMP_C`.

Knowing how to use your own function in pandas can be really useful when doing your own analyses. Here, we will introduce two different approaches for using function in pandas. First, we will see how we can apply the function row-by-row using a `for`-loop and the `DataFrame.iterrows()`-method, and then we will learn how to apply the method to all rows at once using [DataFrame.apply](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

For both of these approaches, we first need to define our temperature conversion function from Fahrenheit to Celsius:

In [ ]:
def fahr_to_celsius(temp_fahrenheit):
    """
    Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)
        
    Returns
    -------
    
    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius and return it
    converted_temp = (temp_fahrenheit - 32) / 1.8
    return converted_temp

**Note:** with such a simple example, we could use the function direcly on a column in the DataFrame in order to conver the values:

If we want to do something more complicated, we need to know how to apply the function row-by-row.

### Iterating over rows

We can iterate over the rows of Pandas DataFrame by using the `iterrows()` -method and use the function one row at a time.


When iterating over the rows in our `DataFrame`, it is noteworthy to understand that the Pandas actually keeps track on the `index` value as well. Hence, the contents of a single row actually contains not only the values, but also the `index` of that row (each row is a pandas Series!). 

- Let's see how `iterrows()` works by printing out the `TEMP` value on each row using a `for`-loop:


In [ ]:
# Iterate over the rows
for idx, row in data.iterrows():
    # Print the index value
    print('Index:', idx)
    
    # Print the row
    print('Temp F:', row["TEMP_F"], "\n")
    
    break

<div class="alert alert-info">

**break**

When developing a for-loop, you don't always need to go trough the whole loop if you just want to test things out. 
[break](https://www.tutorialspoint.com/python/python_break_statement.htm) statement in Python terminates the current loop after the first iteration and we used it here just to test check out the values on the first row.
    
</div>


We can see that the `idx` variable indeed contains the index value at position 0 (the first row) and the `row` variable contains all the data from that given row stored as a pandas `Series`.

- Let's now create an empty column `TEMP_C` for the Celsius temperatures and update the values into that column using the `fahr_to_celsius` function we defined earlier:


In [ ]:
# Create an empty column for the DataFrame where the values will be stored
new_column = "TEMP_C"
data[new_column] = None

# Iterate over the rows 
for idx, row in data.iterrows():
    # Convert the Fahrenheit to Celsius
    celsius = fahr_to_celsius(row['TEMP_F'])
    
    # Update the value of 'Celsius' column with the converted value
    data.at[idx, new_column] = celsius

<div class="alert alert-info">

**.at or .loc?**

Here, you could also use `data.loc[idx, new_column] = celsius` to achieve the same result. 
    
If you only need to access a single value in a DataFrame, [DataFrame.at](https://pandas.pydata.org/pandas-docs/version/0.25/reference/api/pandas.DataFrame.at.html) is faster compared to [DataFrame.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) which is designed for accessing groups of rows and columns.
    
</div>

- Let's see what we have now.

In [ ]:
data.head(10)

Great! Now we have converted our temperatures into Celsius by using our self-made function.

### Applying a function

Pandas DataFrames and Series also have a dedicated method `.apply()` for applying functions on columns (or rows!). When using `.apply()`, we pass the function name (without parenthesis!) as an argument:

**Note:** pay attention which column you are applying the function on! Running this code: `data.apply(fahr_to_celsius)` would not give an error, but the results also don't make much sense.

<div class="alert alert-info">

**Should I use .iterrows() or .apply()?**

We are teaching the `.iterrows()` method because it helps to understand the structure of a DataFrame and the process of looping trough DataFrame rows. However, using `.apply()` is often more efficient in terms of execution time. 
    
    
At this point, the most important thing is that you understand what happens when you are modifying the values in a pandas DataFrame. When doing the course exercises, either of these approaches is ok!
    
</div>

## Parsing dates

We will eventually want to group our data based on month in order to see if April temperatures in 2019 were higher than average. Currently, the date and time information is stored in the column `TIME`:

`YR--MODAHRMN = YEAR-MONTH-DAY-HOUR-MINUTE IN GREENWICH MEAN TIME (GMT)`

Let's have a closer look at the date and time information we have by checking the values in that column, and their data type:

The `TIME` column contains several observations per day (and even several observations per hour). The timestamp for the first observation is `191701010600`, i.e. from 1st of January 1917, and the timestamp for the latest observation is `201910012350` (from last week, by the time of writing this!)

And the information is stored as integer values.

There are several different options for proceeding from here. The bottom line is, that we would want to **aggregate the data on a monthly level**, and in order to do so we need to "label" each row of data based on the month when the record was observed. 
In practice, we could create a new column (or an index), which contains information about the month (including the year, but excluding hours and minutes).

### String slicing

One approach would be to convert the date and time information into character strings and "cut" the needed information from the [string objects](https://docs.python.org/3/tutorial/introduction.html#strings). If we look at the latest time stamp in the data (`201910012350`), you can see that there is a systematic pattern `YEAR-MONTH-DAY-HOUR-MINUTE`. Four first characters represent the year, and six first characters are year + month!

Doing this in pandas requires two steps:
  1. Convert the `TIME` column from `int` into `str` datatype.
  2. Slice the correct range of characters from the character string using [pandas.Series.str.slice()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html)

- Let's convert the time into string. And check that the data type changes:

In [ ]:
# Convert to string


In [ ]:
# SLice the string


# Let's see what we have
print(data.head())

Nice! Now we have "labeled" the rows based on information about day of the year and hour of the day. However, let's have a look at a more clever way of dealing with dates and times..

### datetime

<div class="alert alert-info">

**Pandas datetime**

In pandas, we can convert dates and times into a new data type [datetime](https://docs.python.org/3.7/library/datetime.html) using [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function. First, it is important to understand the structure of the input data in order to avoid erroneous conversions, and that's why we first learned string slicing before introducing the datetime functionalities. 
    
Here is one example of how to convert the `TIME_STR`-column in our data set to datetime:
    
```
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"])

```
   
        
If needed, you can use the `format` parameter to define the output datetime format according to [strftime(format) method](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior). together with `exact=False`, for example like this: 
    
```
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"], format='%Y%m%d%H', exact=False)

```
In this example, `exact=False` drops out minutes and secods, because they are not included in the specified formatting.
 
</div>




In [ ]:
# Convert to datetime


Note: in this case, the data type of the values is `datetime`. 

<div class="alert alert-info">

**Pandas Series datetime properties**
    
There are several methods available for accessing information about the properties of datetime values. Read more from the pandas documentation about [datetime properties](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetime-properties).
</div>

Now, we can extract different time units based on the datetime-column using the [pandas.Series.dt accessor](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html):

We can also combine the datetime functionalities with other methods from pandas. For example, we can check the number of unique years in our input data: 

<div class="alert alert-info">

**TASK:**

- Create two new columns: `YEAR` and `MONTH` based on the date column
</div>

## Aggregating data in Pandas by grouping

Here, we will learn how to use [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) which is a handy method for compressing large amounts of data and computing statistics for subgroups.

Our practical task is to calculate the average temperatures for each month

This can be done by aggregating the data, i.e.:

  1. **grouping the data** based on year and month
  2. Calculating the average for each month (each group) either by using a for-loop or directly from the grouped object
  3. Storing those values into **a new DataFrame** `monthly_data`

Before we start grouping the data, let's once more check how our input data looks like:

In [ ]:
print("number of rows:", len(data))
data.head()

We have quite a few rows of weather data, and several observations per day (and even per hour). **Our goal is to create an aggreated data frame that would have only one row per month!**

Let's **group** our data based on unique year and month combinations

<div class="alert alert-info">

**NOTE:**
    
Here you could also group the data based on the `MONTH_STR` column to achieve the same result:
    
```
# Group the data 
grouped = data.groupby('MONTH_STR')
```
</div>

- Let's see what we have now.

In [ ]:
# What is the type?
print("Type:\n", type(grouped))

# How many?
print("Length:\n", len(grouped))

Okey, interesting. Now we have a new object with type `DataFrameGroupBy`.

<div class="alert alert-info">

**TASK:**

Think: what does the number of groups (length of the grouped object) tell us?
</div>

There are several methods we can use for extracting information from the grouped data. See [documentation for Pandas GroupBy objects](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) for a comprehensive overview. 

**Checking group names:**

In [ ]:
# Check the "names" of each group (uncomment the next row if you want to print out the result!)
#grouped.groups.keys()

**Accessing data for one group:**

- Let's check the contents for a group representing August 2019 (name of that group is `(2019, 4)` if you grouped the data based on datetime columns `YEAR` and `MONTH`). We can get the values of that hour from the grouped object using the `get_group()` -method:

In [ ]:
# Specify the time of the first hour (as text)


# Select the group


# Let's see what we have


Ahaa! As we can see, a single group contains a **DataFrame** with values only for that specific month. Let's check the DataType of this group:

So, one group is a pandas DataFrame! This is really useful, because we can now use all the familiar DataFrame methods for calculating statistics etc for this spesific group. 
We can, for example, calculate the average values for all variables using the statistical functions that we have seen already (e.g. mean, std, min, max, median, etc.).

We can do that by using the `mean()` -function that we already used during the Lesson 5. 

- Let's calculate the mean for following attributes all at once:
   - `DIR`, 
   - `SPEED`, 
   - `GUST`, 
   - `TEMP`, 
   - `TEMP_C`
   - `MONTH` 

In [ ]:
# Specify the columns that will be part of the calculation
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C', 'MONTH']

# Calculate the mean values all at one go


# Let's see what we have


Nice, now we have averaged our data and e.g. the mean Celsius temperature seems to be about right when comparing to the original values above. 

Here we saw how you can access data from a single group. For getting information about all groups (all months)we can a `for` -loop or methods available in the grouped object.

**For-loops and grouped objects:**

When iterating over the groups in our `DataFrameGroupBy` -object
it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the `key` that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

- Let's see how we can iterate over the groups and print the key and the data from a single group (again using `break` to only see what is happening).


In [ ]:
# Iterate over groups

    # Print key and group
    print("Key:\n", key)
    print("\nFirst rows of data in this group:\n", group.head())
    
    # Stop iteration with break command
    break

Okey so from here we can see that the `key` contains the name of the group (year, month).

- Let's see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. I will repeat slightly the earlier steps so that you can see and better understand what is happening.

In [ ]:
# Create an empty DataFrame for the aggregated values


# The columns that we want to aggregate
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C', "MONTH"]

# Iterate over the groups

   # Aggregate the data
   

   # Add the ´key´ (i.e. the date+time information) into the aggregated values
   

   # Append the aggregated values into the DataFrame
   

- Let's see what we have now:

In [ ]:
print(hourly_data)

Awesome! Now we have aggregated our data based on daily averages and we have a new DataFrame called `hourly_data` where all those aggregated values are stored.

**Mean for all groups at once**

We can also achieve the same result by computing the mean of all columns for all groups in the grouped object:

## Detecting warm months

Now, we have aggregated our data on monthly level and all we need to do is to check which years had the warmest April temperatures. A simple approach is to select all aprils from the data, group the data and check which group(s) have the highest mean value:

- select all records that are from April (regardless of the year):

- take a subset of columns that might contain interesting information:

In [ ]:
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C','YEAR', 'MONTH']]

- group by year and month:

- calculate mean for each group:

- check the highest temperature values (sort the data frame in a descending order):

How did April 2019 rank at the Tampere Pirkkala observation station 🌡️? 

## Repeating the data analysis with larger dataset


Finally, let's repeat the data analysis steps above for all the available data we have (!!). First, confirm the path to the **folder** where all the input data are located. 
The idea is, that we will repeat the analysis process for each input file using a (rather long) for loop! Here we have all the main analysis steps with some additional output info - all in one long code cell:

In [ ]:
# Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

# Rename the columns
new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
data = data.rename(columns=new_names)

#Print info about the current input file:
print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
print("NUMBER OF OBSERVATIONS:", len(data))

# Create column
col_name = 'TEMP_C'
data[col_name] = None

# Convert tempetarues from Fahrenheits to Celsius
data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

# Convert TIME to string 
data['TIME_STR'] = data['TIME'].astype(str)

# Parse year and month
data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

# Extract observations for the months of April 
aprils = data[data['MONTH']==4]

# Take a subset of columns
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

# Group by year and month
grouped = aprils.groupby(by=["YEAR", "MONTH"])

# Get mean values for each group
monthly_mean = grouped.mean()

# Print info
print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
print("\n")

We will use the `glob()` function from the module `glob` to list our input files. 

In [ ]:
import glob

<div class="alert alert-info">

**Note**

Note that we're using the \* character as a wildcard, so any file that starts with `data/0` and ends with `txt` will be added to the list of files we will iterate over. We specifically use `data/0` as the starting part of the file names to avoid having our metadata files included in the list!
</div>

In [ ]:
print("Number of files in the list", len(file_list))
print(file_list)

Now, you should have all the relevant file names in a list, and we can loop over the list using a for-loop:

In [ ]:
for fp in file_list:
    print(fp)

In [ ]:
# Repeat the analysis steps for each input file:
for fp in file_list:

    # Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
    data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

    # Rename the columns
    new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
    data = data.rename(columns=new_names)

    #Print info about the current input file:
    print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
    print("NUMBER OF OBSERVATIONS:", len(data))

    # Create column
    col_name = 'TEMP_C'
    data[col_name] = None

    # Convert tempetarues from Fahrenheits to Celsius
    data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

    # Convert TIME to string 
    data['TIME_STR'] = data['TIME'].astype(str)

    # Parse year and month
    data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
    data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

    # Extract observations for the months of April 
    aprils = data[data['MONTH']==4]

    # Take a subset of columns
    aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

    # Group by year and month
    grouped = aprils.groupby(by=["YEAR", "MONTH"])

    # Get mean values for each group
    monthly_mean = grouped.mean()

    # Print info
    print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
    print("\n")

How about now, how did April 2019 rank across different stations?